# Lab 03 · Gemini Proxy Chat

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A Gemini proxy endpoint is created in FastAPI.
- React chat UI components are connected to the proxy.
- API keys remain confined to the backend.

## What will be learned
- Backend proxy patterns for hosted LLMs are practiced.
- Basic chat state management in React is reviewed.
- Environment variable usage is reinforced.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install google-generativeai
```

## Step-by-step tasks
### Step 1: Tour the Gemini service module

Open `ai-web/backend/app/services/gemini.py` and guide students through the
implementation. The file is heavily commented so you can read it nearly line by
line:

```python
"""Service helpers for Gemini-powered features exposed by the FastAPI app."""

from __future__ import annotations

import os
from functools import lru_cache
from typing import List
```

- The module docstring reminds instructors that all Gemini logic should live in
  the service layer.
- `from __future__ import annotations` keeps type hints as strings at runtime so
  FastAPI imports stay lightweight.
- Standard-library imports (`os`, `lru_cache`, `typing.List`) support
  configuration, caching, and parsing.

```python
try:
    import google.generativeai as genai
except ImportError as exc:
    genai = None
    _IMPORT_ERROR = exc
else:
    _IMPORT_ERROR = None
```

- The `try/except` block lets unit tests run without the optional SDK. If the
  import fails, the service stores the exception and raises a friendly error when
  called.

```python
class GeminiServiceError(RuntimeError):
    """Raised when the Gemini helper cannot fulfill a request."""
```

- Custom exceptions make it easy for routers to translate domain failures into
  HTTP 503 responses.

```python
def _require_api_key() -> str:
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        raise GeminiServiceError(
            "GEMINI_API_KEY is not configured. Add it to backend/.env before calling the service."
        )
    return api_key
```

- `_require_api_key` centralizes validation so every caller gets the same error
  message if the environment is missing.

```python
@lru_cache(maxsize=1)
def _configure_client(api_key: str) -> bool:
    if genai is None:
        raise GeminiServiceError(
            "google-generativeai is not installed. Run `pip install google-generativeai` to enable the feature."
        ) from _IMPORT_ERROR

    genai.configure(api_key=api_key)
    return True
```

- `@lru_cache` ensures the expensive `genai.configure` call only runs once per
  process. The guard re-raises the original import error with additional context.

```python
def _parse_outline_lines(raw_outline: str) -> List[str]:
    lines: List[str] = []
    for line in raw_outline.splitlines():
        cleaned = line.strip()
        if not cleaned:
            continue
        cleaned = cleaned.lstrip("-*•0123456789. \t")
        if cleaned:
            lines.append(cleaned)
    return lines
```

- `_parse_outline_lines` sanitizes model output by trimming blank lines and
  stripping bullets/numbers so the frontend receives plain text.

```python
def generate_lesson_outline(topic: str, model: str | None = None) -> dict[str, str | list[str]]:
    normalized_topic = topic.strip()
    if not normalized_topic:
        raise ValueError("Topic must not be empty.")

    api_key = _require_api_key()
    _configure_client(api_key)

    selected_model = model or os.getenv("GEMINI_MODEL", "gemini-2.5-flash")
    try:
        generative_model = genai.GenerativeModel(selected_model)
        prompt = (
            "You are helping an instructor design a web programming lesson. "
            "Return a concise outline with 3-5 bullet points that cover the key "
            "concepts for the topic: "
            f"{normalized_topic}."
        )
        response = generative_model.generate_content(prompt)
        outline_text = getattr(response, "text", "").strip()
    except Exception as exc:
        raw_message = str(exc).strip()
        if raw_message:
            detail = f": {raw_message}"
        else:
            fallback = exc.__class__.__name__
            detail = f": {fallback}"
        raise GeminiServiceError(
            f"Failed to generate lesson outline{detail}."
        ) from exc

    outline = _parse_outline_lines(outline_text) if outline_text else []
    return {"topic": normalized_topic, "outline": outline}
```

- The public function handles validation, client configuration, prompt creation,
  and error translation. Use this walkthrough to emphasize how services encapsulate
  third-party SDK calls while returning simple dictionaries to routers.


#### Reference implementation: `ai-web/backend/app/services/gemini.py`

The complete service module is reproduced below so you can review it in class without running code cells. Highlight how the repository file mirrors each comment and helper shown in the lab walkthrough.

```python
"""Service helpers for Gemini-powered features exposed by the FastAPI app.

The teaching labs encourage a service layer that contains the business logic
for each feature. Routers stay thin while services interact with third-party
SDKs or databases. This module demonstrates that pattern for Gemini requests
and is annotated line-by-line so instructors can explain each operation in
class.
"""

from __future__ import annotations

import os
from functools import lru_cache
from typing import List

# Import the Gemini SDK lazily so unit tests (or classrooms without credentials)
# can still import the module and read through the teaching notes.
try:
    import google.generativeai as genai
except ImportError as exc:  # pragma: no cover - handled during runtime usage.
    genai = None  # type: ignore[assignment]
    _IMPORT_ERROR = exc
else:
    _IMPORT_ERROR = None


class GeminiServiceError(RuntimeError):
    """Raised when the Gemini helper cannot fulfill a request."""


def _require_api_key() -> str:
    """Read the API key from the environment and raise a descriptive error.

    The helper keeps API key access in one place so the error message remains
    consistent every time an instructor demonstrates a misconfigured setup.
    """

    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        raise GeminiServiceError(
            "GEMINI_API_KEY is not configured. Add it to backend/.env before calling the service."
        )
    return api_key


@lru_cache(maxsize=1)
def _configure_client(api_key: str) -> bool:
    """Configure the global Gemini client once per process.

    The return value is a simple boolean so callers can ignore the result and
    focus on the fact that the client has been prepared for use.
    """

    if genai is None:  # pragma: no cover - depends on optional dependency.
        raise GeminiServiceError(
            "google-generativeai is not installed. Run `pip install google-generativeai` to enable the feature."
        ) from _IMPORT_ERROR

    genai.configure(api_key=api_key)
    return True


def _parse_outline_lines(raw_outline: str) -> List[str]:
    """Convert the model response into a clean list of outline bullet points.

    This mirrors what instructors explain in the Lab 03 notebook: Gemini often
    returns Markdown bullets or numbered steps that the UI should present as
    plain text list items.
    """

    lines: List[str] = []
    for line in raw_outline.splitlines():
        cleaned = line.strip()
        if not cleaned:
            continue
        # Remove leading numbering/bullet characters that models often return.
        cleaned = cleaned.lstrip("-*•0123456789. 	")
        if cleaned:
            lines.append(cleaned)
    return lines


def generate_lesson_outline(topic: str, model: str | None = None) -> dict[str, str | list[str]]:
    """Generate a course outline for the requested topic using Gemini.

    Args:
        topic: Instructor-provided lesson topic from the frontend form.
        model: Optional override so the labs can experiment with different
            Gemini releases. Defaults to ``GEMINI_MODEL`` or ``gemini-2.5-flash``.

    Returns:
        Dictionary containing the normalized topic string and a list of outline
        bullet points. The structure matches the response model defined in
        ``app/routers/gemini.py``.

    Raises:
        ValueError: If the topic is empty after trimming whitespace.
        GeminiServiceError: When credentials are missing, the SDK is not
            installed, or the Gemini API reports an error.
    """

    normalized_topic = topic.strip()
    if not normalized_topic:
        raise ValueError("Topic must not be empty.")

    api_key = _require_api_key()
    _configure_client(api_key)  # Cache-aware setup keeps repeated requests fast.

    selected_model = model or os.getenv("GEMINI_MODEL", "gemini-2.5-flash")
    try:
        generative_model = genai.GenerativeModel(selected_model)
        prompt = (
            "You are helping an instructor design a web programming lesson. "
            "Return a concise outline with 3-5 bullet points that cover the key "
            "concepts for the topic: "
            f"{normalized_topic}."
        )
        response = generative_model.generate_content(prompt)
        outline_text = getattr(response, "text", "").strip()
    except Exception as exc:  # pragma: no cover - depends on remote API.
        raw_message = str(exc).strip()
        if raw_message:
            detail = f": {raw_message}"
        else:
            fallback = exc.__class__.__name__
            detail = f": {fallback}"
        raise GeminiServiceError(
            f"Failed to generate lesson outline{detail}."
        ) from exc

    outline = _parse_outline_lines(outline_text) if outline_text else []
    return {"topic": normalized_topic, "outline": outline}
```


### Step 2: Review the Gemini router

Next, inspect `ai-web/backend/app/routers/gemini.py` to see how the service is
exposed to the frontend.

```python
router = APIRouter(prefix="/ai", tags=["ai"])
```

- Namespacing routes under `/ai` keeps AI endpoints grouped together for
  documentation.

```python
class LessonOutlineIn(BaseModel):
    topic: constr(strip_whitespace=True, min_length=1)
```

- Input validation mirrors the service’s `strip()` call so bad requests surface
  as HTTP 422 responses before hitting Gemini.

```python
@router.post("/lesson-outline", response_model=LessonOutlineOut)
def lesson_outline(payload: LessonOutlineIn) -> LessonOutlineOut:
    try:
        result = generate_lesson_outline(payload.topic)
    except ValueError as exc:
        raise HTTPException(status_code=422, detail=str(exc)) from exc
    except GeminiServiceError as exc:
        logger.exception("Gemini lesson outline request failed")
        raise HTTPException(status_code=503, detail=str(exc)) from exc

    return LessonOutlineOut(**result)
```

- The route delegates to the service and translates domain errors into HTTP
  exceptions. Mention the `logger.exception` call so instructors remember to show
  console logs when debugging with the class.


#### Reference implementation: `ai-web/backend/app/routers/gemini.py`

Use this listing to point out how the FastAPI router leans on Pydantic for validation and delegates Gemini-specific work to the service layer.

```python
"""API routes that expose Gemini-backed helpers to the frontend.

The notebook for Lab 03 walks through this module line-by-line, so each section
includes commentary that instructors can echo while teaching the flow.
"""

import logging

from fastapi import APIRouter, HTTPException
from pydantic import BaseModel, constr

from app.services.gemini import GeminiServiceError, generate_lesson_outline

# Prefix the router with /ai so every Gemini-powered endpoint is grouped
# together in the automatically generated FastAPI docs.
router = APIRouter(prefix="/ai", tags=["ai"])


logger = logging.getLogger(__name__)


class LessonOutlineIn(BaseModel):
    """Request schema describing the lesson topic submitted by the frontend."""

    topic: constr(strip_whitespace=True, min_length=1)  # type: ignore[valid-type]


class LessonOutlineOut(BaseModel):
    """Response schema returned to the frontend."""

    topic: str
    outline: list[str]


@router.post("/lesson-outline", response_model=LessonOutlineOut)
def lesson_outline(payload: LessonOutlineIn) -> LessonOutlineOut:
    """Delegate the heavy lifting to the Gemini service layer."""

    try:
        result = generate_lesson_outline(payload.topic)
    except ValueError as exc:
        # Map validation issues (such as an empty topic) to an HTTP 422 so the
        # frontend can display a friendly inline error message.
        raise HTTPException(status_code=422, detail=str(exc)) from exc
    except GeminiServiceError as exc:
        # Log the full stack trace for instructors while returning a concise
        # error payload to the browser.
        logger.exception("Gemini lesson outline request failed")
        raise HTTPException(status_code=503, detail=str(exc)) from exc

    return LessonOutlineOut(**result)
```


### Step 3: Demonstrate the Gemini lesson-outline feature

Frame the frontend as a Vite-powered React single-page app before you run the demo. This gives the user an architectural mental model and makes each source file easier to map to what appears in the browser.

- **Vite + `index.html`** – Vite serves `ai-web/frontend/index.html`, which mounts a root `<div>` that React uses as its rendering target. Vite also injects environment variables under the `import.meta.env` namespace so the browser knows where to send API calls.
- **Entry point (`src/main.jsx`)** – Calls `ReactDOM.createRoot(...).render(...)` to boot the React tree. `React.StrictMode` keeps runtime checks enabled in development and mirrors the scaffolding students see in earlier labs.
- **App shell (`src/App.jsx`)** – Imports feature hooks and components, calls the hooks to obtain stateful values, and spreads those values into the JSX. This file proves the pattern of "compose with hooks, render with components" that repeats across labs.

Then drill into the Gemini-specific pieces so the user can narrate how data flows from the form to FastAPI and back:

1. **Hook (`features/gemini/hooks/useLessonOutlineForm.js`)** – Owns the React state for the controlled input (`topic`), derived output (`outline`), fetch status (`loading`), and any backend failure (`error`). The exported `handleSubmit` function prevents the default form post, trims whitespace, rejects empty strings, and calls the shared `post()` helper. The success branch stores the outline array; the error branch normalises FastAPI error shapes so the component can display readable feedback.
2. **HTTP helper (`src/lib/api.js`)** – Wraps `fetch` so every POST request inherits the same JSON headers, environment-aware base URL, and descriptive error handling. Highlight the `post()` function signature (`path`, `body`) and how it throws an Error when the response status is outside the 200 range.
3. **Component (`features/gemini/components/LessonOutlineForm.jsx`)** – Receives the hook output as props and focuses purely on markup. The JSX uses the controlled `topic` value, calls `setTopic` on every keystroke, disables the submit button while `loading` is `true`, prints the error paragraph when `error` has content, and renders an ordered list by mapping the `outline` array. Because the component is stateless, it can be reused in other demos with different hooks.

Conclude by showing the form in the running Vite dev server. Narrate each UI state change (loading spinner text, validation message, outline rendering) as you submit topics so the user understands the entire frontend feedback loop.


#### Reference implementation: Gemini lesson outline UI

Revisit the React feature modules below. They already exist in the repository and mirror the workflow described in this notebook.

```jsx
// File: ai-web/frontend/src/App.jsx
import { EchoForm } from './features/echo/components/EchoForm';
import { useEchoForm } from './features/echo/hooks/useEchoForm';
import { LessonOutlineForm } from './features/gemini/components/LessonOutlineForm';
import { useLessonOutlineForm } from './features/gemini/hooks/useLessonOutlineForm';

function App() {
  const echoForm = useEchoForm();
  const lessonOutlineForm = useLessonOutlineForm();

  return (
    <main style={{ padding: 24, display: 'grid', gap: 32 }}>
      <header style={{ display: 'grid', gap: 8 }}>
        <h1>AI in Web Programming Demos</h1>
        <p>
          FastAPI and React layers
          evolve together. Each section mirrors the workflow documented in the
          instructor guide.
        </p>
      </header>

      <section style={{ display: 'grid', gap: 16 }}>
        <h2>Retrying echo service</h2>
        <EchoForm {...echoForm} />
      </section>

      <section style={{ display: 'grid', gap: 16 }}>
        <h2>Gemini lesson outline builder</h2>
        <LessonOutlineForm {...lessonOutlineForm} />
      </section>
    </main>
  );
}

export default App;
```

```jsx
// File: ai-web/frontend/src/features/gemini/components/LessonOutlineForm.jsx
export function LessonOutlineForm({ topic, setTopic, outline, loading, error, handleSubmit }) {
  return (
    <form onSubmit={handleSubmit} style={{ display: 'grid', gap: 12 }}>
      <p>
        Generate a quick lesson outline powered by Gemini. Provide a topic, submit
        the form, and discuss the generated talking points with your class.
      </p>

      <label style={{ display: 'grid', gap: 4 }}>
        <span>Lesson topic</span>
        <input
          type="text"
          value={topic}
          onChange={(event) => setTopic(event.target.value)}
          placeholder="e.g. Building resilient web APIs"
          disabled={loading}
          required
        />
      </label>

      <button type="submit" disabled={loading || !topic.trim()}>
        {loading ? 'Generating outline…' : 'Generate outline'}
      </button>

      {error && (
        <p style={{ color: 'crimson' }}>
          {error}. Confirm the backend has access to <code>GEMINI_API_KEY</code>.
        </p>
      )}

      {outline.length > 0 && (
        <ol>
          {outline.map((item, index) => (
            <li key={`${item}-${index}`}>{item}</li>
          ))}
        </ol>
      )}
    </form>
  );
}
```

```javascript
// File: ai-web/frontend/src/features/gemini/hooks/useLessonOutlineForm.js
import { useCallback, useState } from 'react';

import { post } from '../../../lib/api';

const ENDPOINT = '/ai/lesson-outline';

export function useLessonOutlineForm() {
  const [topic, setTopic] = useState('');
  const [outline, setOutline] = useState([]);
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState(null);

  const handleSubmit = useCallback(
    async (event) => {
      event.preventDefault();
      const trimmedTopic = topic.trim();
      if (!trimmedTopic) {
        setError('Please enter a topic.');
        setOutline([]);
        return;
      }

      setLoading(true);
      setError(null);

      try {
        const response = await post(ENDPOINT, { topic: trimmedTopic });
        setOutline(Array.isArray(response.outline) ? response.outline : []);
      } catch (err) {
        setOutline([]);
        const detailMessage =
          typeof err?.detail === 'string'
            ? err.detail
            : err instanceof Error && err.message
              ? err.message
              : 'Unknown error';
        setError(detailMessage);
      } finally {
        setLoading(false);
      }
    },
    [topic]
  );

  return {
    topic,
    setTopic,
    outline,
    loading,
    error,
    handleSubmit
  };
}
```


## Validation / acceptance checks
```bash
# locally
export GEMINI_API_KEY="<your real key>"
uvicorn app.main:app --reload
curl -X POST http://localhost:8000/ai/lesson-outline   -H 'Content-Type: application/json'   -d '{"topic":"State management in React"}'
```
- Response JSON includes the normalized topic and an `outline` array.
- Frontend form displays bullet points that match the backend payload.
- If `GEMINI_API_KEY` or the SDK import is missing, the API responds with HTTP 503
  and surfaces the descriptive error message from `GeminiServiceError`.


## Homework / extensions
- Streaming responses are researched for future enhancements.
- Chat history persistence is sketched for the next lab.